In [1]:
# Install in Google Colab
try:
    import google.colab
    !pip install -q -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ quantem-widget
except ImportError:
    pass  # Not in Colab, skip

In [2]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

env: ANYWIDGET_HMR=1


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show3d/show3d_load_png_folder.ipynb)
# Show3D — Load PNG Folder
Generate synthetic diffraction patterns as PNG files, then load the folder as a stack with `Show3D.from_folder(path, file_type="png")`.

## 1. Generate synthetic diffraction patterns
Each frame simulates a polycrystalline electron diffraction pattern at a slightly different beam tilt, so Bragg spots shift across frames — realistic for a tilt series or precession scan.

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import tempfile
from pathlib import Path
import numpy as np
import torch
from PIL import Image
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")
# --- Parameters ---
n_frames = 12
size = 256
n_spots = 24       # Bragg spots per ring
n_rings = 3        # concentric Debye-Scherrer rings
spot_sigma = 3.0   # Bragg spot width (px)
tilt_amplitude = 8  # max tilt shift (px)
# --- Coordinate grid on GPU ---
row = torch.arange(size, device=device, dtype=torch.float32)
col = torch.arange(size, device=device, dtype=torch.float32)
rr, cc = torch.meshgrid(row, col, indexing="ij")  # (size, size)
center = size / 2.0
tilt_angles = torch.linspace(0, 2 * torch.pi * (1 - 1 / n_frames), n_frames, device=device)
# --- Generate Bragg spot positions for each ring ---
ring_radii = torch.tensor([35.0, 60.0, 90.0], device=device)
spot_angles = torch.linspace(0, 2 * torch.pi * (1 - 1 / n_spots), n_spots, device=device)
# Precompute base spot positions: (n_rings * n_spots, 2)
base_rows = []
base_cols = []
for ring_idx in range(n_rings):
    r = ring_radii[ring_idx]
    base_rows.append(center + r * torch.sin(spot_angles + ring_idx * 0.3))
    base_cols.append(center + r * torch.cos(spot_angles + ring_idx * 0.3))
base_spot_rows = torch.cat(base_rows)  # (n_total_spots,)
base_spot_cols = torch.cat(base_cols)
# Varying intensity per spot (some brighter than others)
spot_intensities = 0.5 + 0.5 * torch.rand(len(base_spot_rows), device=device)
# --- Render frames ---
frames = torch.zeros(n_frames, size, size, device=device)
for f in range(n_frames):
    # Beam tilt shifts the pattern
    tilt_row = tilt_amplitude * torch.sin(tilt_angles[f])
    tilt_col = tilt_amplitude * torch.cos(tilt_angles[f])
    shifted_rows = base_spot_rows + tilt_row  # (n_spots_total,)
    shifted_cols = base_spot_cols + tilt_col
    # Vectorized Gaussian spots: broadcast (size, size, 1) vs (1, 1, n_spots)
    dr = rr.unsqueeze(-1) - shifted_rows.view(1, 1, -1)  # (size, size, n_spots)
    dc = cc.unsqueeze(-1) - shifted_cols.view(1, 1, -1)
    gauss = torch.exp(-(dr**2 + dc**2) / (2 * spot_sigma**2))
    # Weighted sum across all spots
    frame = (gauss * spot_intensities.view(1, 1, -1)).sum(dim=-1)
    # Central beam (bright)
    dist_sq = (rr - center - tilt_row)**2 + (cc - center - tilt_col)**2
    frame += 3.0 * torch.exp(-dist_sq / (2 * 6.0**2))
    # Background: gentle radial falloff + noise
    radial = torch.sqrt((rr - center)**2 + (cc - center)**2)
    frame += 0.05 * torch.exp(-radial / 80.0)
    frames[f] = frame
# Add Poisson-like noise on CPU (torch.poisson unreliable on MPS)
frames_np = frames.cpu().numpy()
frames_np = np.clip(frames_np, 0, None)
scale = 500.0
frames_np = np.random.poisson(frames_np * scale).astype(np.float32) / scale
print(f"Generated {n_frames} frames: shape={frames_np.shape}, range=[{frames_np.min():.3f}, {frames_np.max():.3f}]")

Using device: mps
Generated 12 frames: shape=(12, 256, 256), range=[0.000, 3.226]


## 2. Save as PNG files
Each frame is saved as a 16-bit grayscale PNG to preserve dynamic range (8-bit would clip the Bragg peaks).

In [4]:
# Save to a temp folder as 16-bit PNGs
png_folder = Path(tempfile.mkdtemp(prefix="diffraction_pngs_"))
tilt_deg = np.linspace(-6, 6, n_frames)
for i in range(n_frames):
    frame = frames_np[i]
    # Normalize to 16-bit range
    fmin, fmax = frame.min(), frame.max()
    if fmax > fmin:
        normalized = ((frame - fmin) / (fmax - fmin) * 65535).astype(np.uint16)
    else:
        normalized = np.zeros_like(frame, dtype=np.uint16)
    img = Image.fromarray(normalized, mode="I;16")
    img.save(png_folder / f"tilt_{tilt_deg[i]:+05.1f}deg.png")
print(f"Saved {n_frames} PNGs to: {png_folder}")
for p in sorted(png_folder.iterdir()):
    print(f"  {p.name}  ({p.stat().st_size / 1024:.0f} KB)")

Saved 12 PNGs to: /var/folders/xz/h9zytq2x1kgcckdf20y6hn4w0000gn/T/diffraction_pngs_dovwdacx
  tilt_+00.5deg.png  (71 KB)
  tilt_+01.6deg.png  (72 KB)
  tilt_+02.7deg.png  (69 KB)
  tilt_+03.8deg.png  (71 KB)
  tilt_+04.9deg.png  (69 KB)
  tilt_+06.0deg.png  (68 KB)
  tilt_-00.5deg.png  (74 KB)
  tilt_-01.6deg.png  (74 KB)
  tilt_-02.7deg.png  (72 KB)
  tilt_-03.8deg.png  (68 KB)
  tilt_-04.9deg.png  (73 KB)
  tilt_-06.0deg.png  (70 KB)


/var/folders/xz/h9zytq2x1kgcckdf20y6hn4w0000gn/T/ipykernel_72069/2581349230.py:13: DeprecationWarning: 'mode' parameter for changing data types is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(normalized, mode="I;16")


## 3. Load the PNG folder with Show3D
`from_folder()` reads all files of the given type in sorted order, converts to float32, and displays as a stack. Use log scale to bring out the faint outer Bragg rings.

In [ ]:
import quantem.widget
from quantem.widget import Show3D
w = Show3D.from_folder(
    png_folder,
    file_type="png",
    title="Diffraction Tilt Series (from PNGs)",
)
w
print(f"quantem.widget {quantem.widget.__version__}")

## 4. Inspect and interact
Check the loaded stack shape and use the programmatic API.

In [6]:
w.summary()

Diffraction Tilt Series (from PNGs)
════════════════════════════════
Stack:    12×256×256
Frame:    6/11 [tilt_-00.5deg.png]
Data:     min=0  max=6.554e+04  mean=1531
Display:  magma | manual contrast | linear
Playback: 5.0 fps | loop=on | reverse=off | boomerang=off


In [7]:
# Place an ROI on the central beam to track intensity across tilt
w.set_roi(row=128, col=128, radius=15)
w.roi_active

True

## 5. Alternative: `from_path()` auto-detects files vs folders
`from_path()` works for both single files and folders. For folders, `file_type` is required.

# Equivalent to from_png_folder — from_path auto-detects directory
w2 = Show3D.from_path(png_folder, file_type="png", title="via from_path()", cmap="gray")
w2